In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten, LSTM
from keras.layers import Embedding
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.layers import Input
from keras.layers.merge import Concatenate

Using TensorFlow backend.


In [2]:
import tensorflow as tf

## 1. Import data: Combine & Merge Tables

In [302]:
fd = pd.read_csv("full_data.csv")
fd = fd.iloc[:,[0,1,3,4,5,9,10]]
print(fd.shape)
fd.head()

(48979, 7)


,product_id,brand,product_full_name,description,brand_category,brand_canonical_url,details
0,01DSE9TC2DQXDG6GWKW9NMJ416,Banana Republic,Ankle-Strap Pump,"A modern pump, in a rounded silhouette with an...",Unknown,https://bananarepublic.gap.com/browse/product....,"A modern pump, in a rounded silhouette with an..."
1,01DSE9SKM19XNA6SJP36JZC065,Banana Republic,Petite Tie-Neck Top,Dress it down with jeans and sneakers or dress...,Unknown,https://bananarepublic.gap.com/browse/product....,Dress it down with jeans and sneakers or dress...
2,01DSJX8GD4DSAP76SPR85HRCMN,Loewe,52MM Padded Leather Round Sunglasses,Padded leather covers classic round sunglasses.,JewelryAccessories/SunglassesReaders/RoundOval...,https://www.saksfifthavenue.com/loewe-52mm-pad...,100% UV protection Case and cleaning cloth inc...
3,01DSJVKJNS6F4KQ1QM6YYK9AW2,Converse,Baby's & Little Kid's All-Star Two-Tone Mid-To...,The iconic mid-top design gets an added dose o...,"JustKids/Shoes/Baby024Months/BabyGirl,JustKids...",https://www.saksfifthavenue.com/converse-babys...,Canvas upper Round toe Lace-up vamp SmartFOAM ...
4,01DSK15ZD4D5A0QXA8NSD25YXE,Alexander McQueen,64MM Rimless Sunglasses,Hexagonal shades offer a rimless view with int...,JewelryAccessories/SunglassesReaders/RoundOval,https://www.saksfifthavenue.com/alexander-mcqu...,100% UV protection Gradient lenses Adjustable ...


In [303]:
tag = pd.read_excel("tagged.xlsx")
extra_tag = pd.read_csv("additional_tags.csv")
all_tag = pd.concat([tag,extra_tag])

In [304]:
merge = fd.merge(all_tag,left_on='product_id',right_on='product_id',how="inner")
merge.shape

(159013, 10)

In [305]:
merge.head()

,product_id,brand,product_full_name,description,brand_category,brand_canonical_url,details,product_color_id,attribute_name,attribute_value
0,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise. 31"" inseam; 14"" leg o...",01E5ZXP5JCREDC7WJVMWHK5Q40,materialclothing,linenblend
1,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise. 31"" inseam; 14"" leg o...",01E5ZXP5JCREDC7WJVMWHK5Q40,materialclothing,cottonblend
2,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise. 31"" inseam; 14"" leg o...",01E5ZXP5JCREDC7WJVMWHK5Q40,style,modern
3,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise. 31"" inseam; 14"" leg o...",01E5ZXP5JCREDC7WJVMWHK5Q40,style,businesscasual
4,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise. 31"" inseam; 14"" leg o...",01E5ZXP5JCREDC7WJVMWHK5Q40,style,classic


## 2. Prepare Data

#### Filter out records for Attribute "Occasion"

In [63]:
data = merge[merge["attribute_name"]=="occasion"]
for i in data.columns:
    data[i] = data[i].str.lower()
print(data.shape)
data.head()

(21263, 10)


/Users/bingruxue/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,product_id,brand,name,description,brand_category,brand_canonical_url,details,product_color_id,attribute_name,attribute_value
2,01dtjcerf6f4nrz2wsjffa1eys,theory,teah stretch-silk camisole,"beige stretch-silk slips on 93% silk, 7% spand...",clothing / tops / tanks and camis,https://www.net-a-porter.com/us/en/product/119...,"fits true to size, take your normal size\ncut ...",01dtjcergehdmq5fajqs2pz6nj,occasion,weekend
3,01dtjcerf6f4nrz2wsjffa1eys,theory,teah stretch-silk camisole,"beige stretch-silk slips on 93% silk, 7% spand...",clothing / tops / tanks and camis,https://www.net-a-porter.com/us/en/product/119...,"fits true to size, take your normal size\ncut ...",01dtjcergehdmq5fajqs2pz6nj,occasion,work
12,01dtjcerf6f4nrz2wsjffa1eys,theory,teah stretch-silk camisole,"beige stretch-silk slips on 93% silk, 7% spand...",clothing / tops / tanks and camis,https://www.net-a-porter.com/us/en/product/119...,"fits true to size, take your normal size\ncut ...",01dtjcergehdmq5fajqs2pz6nj,occasion,day to night
15,01dtjcerf6f4nrz2wsjffa1eys,theory,teah stretch-silk camisole,"beige stretch-silk slips on 93% silk, 7% spand...",clothing / tops / tanks and camis,https://www.net-a-porter.com/us/en/product/119...,"fits true to size, take your normal size\ncut ...",01dtjcergehdmq5fajqs2pz6nj,occasion,night out
30,01dtjcerf6f4nrz2wsjffa1eys,theory,teah stretch-silk camisole,"beige stretch-silk slips on 93% silk, 7% spand...",clothing / tops / tanks and camis,https://www.net-a-porter.com/us/en/product/119...,"fits true to size, take your normal size\ncut ...",01dtjcergehdmq5fajqs2pz6nj,occasion,work


In [64]:
# Check unique attribute value for occasion
data['attribute_value'].unique()

array(['weekend', 'work', 'day to night', 'night out', 'nightout',
       'daytonight', 'vacation', 'workout', 'coldweather'], dtype=object)

In [65]:
def regex_cleaning(value):
    new_value = re.sub("nightout",'night out', value)
    new_value1 = re.sub("daytonight", 'day to night', new_value)
    new_value2 = re.sub("coldweather", 'cold weather', new_value1)
    return new_value2
data['attribute_value'] = data['attribute_value'].apply(regex_cleaning)

/Users/bingruxue/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [66]:
data['attribute_value'].unique()
# Check again, we got in total 7 different occasion categories

array(['weekend', 'work', 'day to night', 'night out', 'vacation',
       'workout', 'cold weather'], dtype=object)

In [67]:
# Create entity group: Combine product_id and product_color_id
data['id'] = data['product_id']+data['product_color_id']

/Users/bingruxue/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [68]:
# Check whether we have duplicate records
data.groupby('id')['attribute_value'].count()

id
01dpc9gstt72khnn0mndnkh7rd01dpc9gstzypv54n9b1tbb6h1x     4
01dpc9gstt72khnn0mndnkh7rd01dpc9gtp1qdcw3exqttr6ad94     4
01dpc9gstt72khnn0mndnkh7rd01dpc9gvg1zn8rrax77s27jrzs     4
01dpcb2keavxxkfvm7fxbne4vy01dpcb2kef1cfywfwa97d3vn11    10
01dpcb2keavxxkfvm7fxbne4vy01dpcb2mmtpctv3rnjv2bbaf28    10
                                                        ..
01e6078g3gratf2c96vkyywsgd01e6078g42kzpg57ncp4yg1tbb     1
01e6078g3gratf2c96vkyywsgd01e6078m3mmsaetkp9pd5dprr2     1
01e6079dg58yw9k78d57c6j2y101e6079dgrr4chkcpkny96g15p     3
01e6079qfkh4hpzfq31t6wdrrx01e6079qg5gtp5jyrcwgrwy4zc     1
01e607bhrqajdz76mjfn7rprk101e607bhsbcwh034nx5tmjh3z7     2
Name: attribute_value, Length: 5411, dtype: int64

In [69]:
# remove duplicate records
data.drop_duplicates(subset =["id",'attribute_name','attribute_value'],
                     keep = "first",inplace = True) 
data.shape

/Users/bingruxue/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(12625, 11)

#### Create dummies for response variable

In [70]:
dummies = pd.get_dummies(data['attribute_value'])
dummies.head()

,cold weather,day to night,night out,vacation,weekend,work,workout
2,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0
12,0,1,0,0,0,0,0
15,0,0,1,0,0,0,0
94,0,0,1,0,0,0,0


In [71]:
data = pd.concat([data, dummies], axis=1, sort=False)
data.head()

,product_id,brand,name,description,brand_category,brand_canonical_url,details,product_color_id,attribute_name,attribute_value,id,cold weather,day to night,night out,vacation,weekend,work,workout
2,01dtjcerf6f4nrz2wsjffa1eys,theory,teah stretch-silk camisole,"beige stretch-silk slips on 93% silk, 7% spand...",clothing / tops / tanks and camis,https://www.net-a-porter.com/us/en/product/119...,"fits true to size, take your normal size\ncut ...",01dtjcergehdmq5fajqs2pz6nj,occasion,weekend,01dtjcerf6f4nrz2wsjffa1eys01dtjcergehdmq5fajqs...,0,0,0,0,1,0,0
3,01dtjcerf6f4nrz2wsjffa1eys,theory,teah stretch-silk camisole,"beige stretch-silk slips on 93% silk, 7% spand...",clothing / tops / tanks and camis,https://www.net-a-porter.com/us/en/product/119...,"fits true to size, take your normal size\ncut ...",01dtjcergehdmq5fajqs2pz6nj,occasion,work,01dtjcerf6f4nrz2wsjffa1eys01dtjcergehdmq5fajqs...,0,0,0,0,0,1,0
12,01dtjcerf6f4nrz2wsjffa1eys,theory,teah stretch-silk camisole,"beige stretch-silk slips on 93% silk, 7% spand...",clothing / tops / tanks and camis,https://www.net-a-porter.com/us/en/product/119...,"fits true to size, take your normal size\ncut ...",01dtjcergehdmq5fajqs2pz6nj,occasion,day to night,01dtjcerf6f4nrz2wsjffa1eys01dtjcergehdmq5fajqs...,0,1,0,0,0,0,0
15,01dtjcerf6f4nrz2wsjffa1eys,theory,teah stretch-silk camisole,"beige stretch-silk slips on 93% silk, 7% spand...",clothing / tops / tanks and camis,https://www.net-a-porter.com/us/en/product/119...,"fits true to size, take your normal size\ncut ...",01dtjcergehdmq5fajqs2pz6nj,occasion,night out,01dtjcerf6f4nrz2wsjffa1eys01dtjcergehdmq5fajqs...,0,0,1,0,0,0,0
94,01dvpbj6464ykygvae0a1hmkgn,alexander wang,layered velvet mini dress,black velvet concealed hook and zip fastening ...,clothing / dresses / mini,https://www.net-a-porter.com/us/en/product/120...,"fits true to size, take your normal size \ndes...",01dvpbj64g517982dcfq9wsfn0,occasion,night out,01dvpbj6464ykygvae0a1hmkgn01dvpbj64g517982dcfq...,0,0,1,0,0,0,0


In [216]:
#distribution of each occasion attribute value
data['attribute_value'].value_counts()

weekend         4159
day to night    3804
work            1872
night out       1210
vacation         973
cold weather     348
workout          259
Name: attribute_value, dtype: int64

In [84]:
# A same product may belongs to multiple occasions
# Therefore, we sum dummies together for each product
response = df.groupby('id').agg({'cold weather':'sum','day to night':'sum',
                                 'night out':'sum','vacation':'sum',
                                 'weekend':'sum','work':'sum','workout':'sum'}).reset_index()
print(response.shape)
response.head()

(5399, 8)


,id,cold weather,day to night,night out,vacation,weekend,work,workout
0,01dpc9gstt72khnn0mndnkh7rd01dpc9gstzypv54n9b1t...,0,1,0,0,0,1,0
1,01dpc9gstt72khnn0mndnkh7rd01dpc9gtp1qdcw3exqtt...,0,1,0,0,0,1,0
2,01dpc9gstt72khnn0mndnkh7rd01dpc9gvg1zn8rrax77s...,0,1,0,0,0,1,0
3,01dpcb2keavxxkfvm7fxbne4vy01dpcb2kef1cfywfwa97...,0,1,0,0,1,1,0
4,01dpcb2keavxxkfvm7fxbne4vy01dpcb2mmtpctv3rnjv2...,0,1,0,0,1,1,0


In [85]:
combine = data.iloc[:,:11].merge(response,left_on='id',right_on='id')

In [86]:
combine.drop_duplicates(subset =["id"],keep = "first",inplace = True) 
combine.shape

(5399, 18)

In [91]:
combine = combine.set_index('id')

#### Preprocess Feature columns

In [92]:
# drop records with both null value in description and details
combine.dropna(subset=["description", "details"], how='all',inplace =True)

In [93]:
combine.isnull().sum()

product_id               0
brand                    0
name                     0
description            380
brand_category         337
brand_canonical_url      0
details                397
product_color_id         0
attribute_name           0
attribute_value          0
cold weather             0
day to night             0
night out                0
vacation                 0
weekend                  0
work                     0
workout                  0
dtype: int64

In [94]:
# Combine details and description into one "Text" column
combine= combine.replace(np.nan, 'UNKNOWN_TOKEN', regex=True)
combine['details'] = combine['details'].str.replace("\n", "")
combine['text'] = combine['description']+' '+combine['details']

In [36]:
# Preprocess for Text info
import spacy
nlp = spacy.load('en_core_web_md')

def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    
    #remove stopwords and do lemmatization
    doc = nlp(sentence)
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    
    return " ".join(tokens)

In [95]:
combine['text'] = combine['text'].apply(preprocess_text)
combine['name'] = combine['name'].apply(preprocess_text)
combine['brand_category'] = combine['brand_category'].apply(preprocess_text)

In [38]:
def preprocess_url(url):
    # Remove http//:www.
    url = re.sub('https://www.', '', url)
    # Remove .com
    url = re.sub('.com', '', url)
    # Remove separator "/"
    url = re.sub('/', ' ', url)
    # Remove separator "-"
    url = re.sub('-', ' ', url)
    # Remove numbers
    url = re.sub(r'[0-9]+', ' ', url)
    # Remove Single character
    url = re.sub(r"\s+[a-zA-Z]\s+", ' ', url)
    # Removing multiple spaces
    url = re.sub(r'\s+', ' ', url)
    # Remove stopwords and do lemmatization
    doc = nlp(url)
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    
    return " ".join(tokens)

In [96]:
combine['brand_canonical_url'] = combine['brand_canonical_url'].apply(preprocess_url)

In [97]:
# Combine all brand info together
combine['brand_info'] = combine['brand']+' '+combine['name']+' '+\
                        combine['brand_category']+' '+combine['brand_canonical_url']

In [98]:
# Filter out useful columns
df = combine.loc[:,["text","brand_info","cold weather","day to night","night out",
                 "vacation","weekend","work","workout"]]
df.head()

,text,brand_info,cold weather,day to night,night out,vacation,weekend,work,workout
id,,,,,,,,,
01dtjcerf6f4nrz2wsjffa1eys01dtjcergehdmq5fajqs2pz6nj,beige stretch silk slip silk spandex dry clean...,theory teah stretch silk camisole clothing top...,0,1,1,0,1,1,0
01dvpbj6464ykygvae0a1hmkgn01dvpbj64g517982dcfq9wsfn0,black velvet conceal hook zip fastening viscos...,alexander wang layer velvet mini dress clothin...,0,0,1,0,1,0,0
01dpetns8db4mar29er6esa7xa01dpetnv97m40yppkjfh24gmqw,accordion pleat flattering midi length faux le...,j.crew faux leather pleat midi skirt skirt jcr...,0,1,0,0,0,1,0
01dpetns8db4mar29er6esa7xa01dpetns8jn5m4nmqay31d7v8n,accordion pleat flattering midi length faux le...,j.crew faux leather pleat midi skirt skirt jcr...,0,1,0,0,0,1,0
01dt50zst2px9c1kkza4p73q5q01dt50zst833away3pd7ggxyjj,heel measure approximately mm inch snake effec...,alexander wang parker logo embellish snake eff...,0,1,1,0,1,1,0


In [100]:
# check one product
df[df.index=="01dtjcerf6f4nrz2wsjffa1eys01dtjcergehdmq5fajqs2pz6nj"]

,text,brand_info,cold weather,day to night,night out,vacation,weekend,work,workout
id,,,,,,,,,
01dtjcerf6f4nrz2wsjffa1eys01dtjcergehdmq5fajqs2pz6nj,beige stretch silk slip silk spandex dry clean...,theory teah stretch silk camisole clothing top...,0,1,1,0,1,1,0


____

## 3. Embedding Model: Description & Detail

In [188]:
docs = df['text']
labels = df.iloc[:,2:]
X = docs
y = labels.values

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3,random_state=42)

In [189]:
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(X_train)

X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1
vocab_size

5668

In [190]:
from keras.preprocessing.text import text_to_word_sequence
from typing import List
def get_max_token_length_per_doc(docs: List[List[str]])-> int:
    return max(list(map(lambda x: len(x.split()), docs)))
max_length = get_max_token_length_per_doc(docs)
max_length

165

In [191]:
X_train_pad = pad_sequences(X_train_token, padding='post', maxlen=max_length)
X_test_pad = pad_sequences(X_test_token, padding='post', maxlen=max_length)

In [192]:
X_train_pad.shape

(3779, 165)

In [193]:
# using GloVe word embeddings to convert text inputs 
# to their numeric counterparts.

def load_glove_vectors():
    embeddings_index = {}
    with open('glove.6B.100d.txt') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index

embeddings_index = load_glove_vectors()

Loaded 400000 word vectors.


In [194]:
EMBEDDING_SIZE = 100
embedding_matrix = zeros((vocab_size, EMBEDDING_SIZE))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

In [195]:
# Predictive model: LSTM
deep_inputs = Input(shape=(max_length,))
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(deep_inputs)
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(7, activation='sigmoid')(LSTM_Layer_1)
model_lstm = Model(inputs=deep_inputs, outputs=dense_layer_1)

model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [196]:
model_lstm.fit(X_train_pad, y_train, epochs=5, verbose=1)

Epoch 1/5
3779/3779 [==============================] - 12s 3ms/step - loss: 0.4818 - acc: 0.8005
Epoch 2/5
3779/3779 [==============================] - 11s 3ms/step - loss: 0.4630 - acc: 0.8018
Epoch 3/5
3779/3779 [==============================] - 13s 3ms/step - loss: 0.4626 - acc: 0.8018
Epoch 4/5
3779/3779 [==============================] - 13s 3ms/step - loss: 0.4627 - acc: 0.8018
Epoch 5/5
3779/3779 [==============================] - 13s 3ms/step - loss: 0.4622 - acc: 0.8018


In [197]:
# Predict on test dataset
score = model_lstm.evaluate(X_test_pad, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

1620/1620 [==============================] - 2s 1ms/step
Test Score: 0.4632750224184107
Test Accuracy: 0.8000000715255737


In [198]:
cols_target = ["cold weather","day to night","night out","vacation","weekend","work","workout"]
pred = model_lstm.predict(X_test_pad)
embedding_result = pd.DataFrame(data= pred, columns = cols_target,index=X_test.index)
embedding_result

,cold weather,day to night,night out,vacation,weekend,work,workout
id,,,,,,,
01e4ed7cp5h8mjkn93n1n30w8d01e4ed7cpdfeppkcq7dtkeet08,0.054942,0.691541,0.243082,0.182952,0.789172,0.336169,0.042561
01dtatf91fkbx2rw9k53jd1dq901dtatf91qzh744fekqw5z364e,0.054942,0.691541,0.243082,0.182952,0.789172,0.336169,0.042561
01e2kxsgamtvks4rrnxmymygzz01e2kxsznrcmyvgnjehk5sawf7,0.054942,0.691541,0.243082,0.182952,0.789172,0.336169,0.042561
01e4eaztq7jc3chps1bs500tdb01e4eaztqhg69v9344gagstty9,0.054942,0.691541,0.243082,0.182952,0.789172,0.336169,0.042561
01e2kzsh4mdqmrd79yx26vrz1f01e2kzsh4zmzcd1xhyxhtqt3z0,0.054942,0.691541,0.243082,0.182952,0.789172,0.336169,0.042561
...,...,...,...,...,...,...,...
01e2m40trb3jfmw8yqrm9kwqs901e2m42g19qp297dh2c50eat3f,0.054942,0.691541,0.243082,0.182952,0.789172,0.336169,0.042561
01e1jkxpbj02k4e8132x4x8w2c01e1jkxpbxkrqgb2mmbgh4wca8,0.054942,0.691541,0.243082,0.182952,0.789172,0.336169,0.042561
01e4rtwhb4zaagswpfd3awc4za01e4rtwhbe5mw2133e68x690gg,0.054942,0.691541,0.243082,0.182952,0.789172,0.336169,0.042561


____

## 4. Vectorization Model: Brand, Name, URL, Brand Category

In [199]:
info = df['brand_info']
labels = df.iloc[:,2:]
X = info
y = labels

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3,random_state=42)

In [200]:
## TF-IDF Vectorizer
train_X = list(X_train.values)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=3, 
                             max_df=0.9,
                             token_pattern=r'\w{3,}',
                             stop_words='english')
train_vector = vectorizer.fit_transform(train_X)
tfidf_df = pd.DataFrame(train_vector.toarray(), columns=vectorizer.get_feature_names())
tfidf_df

,accessory,ace,acg,achille,acid,adidas,adidas_originals,aeyde,aeydē,agence,...,york,zadig,zebra,zella,zena,zeynep,zimmermann,zip,zoom,étoile
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3774,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3775,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [201]:
test_vector = vectorizer.transform(X_test)

In [202]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression(C=12.0)
cols_target = ["cold weather","day to night","night out","vacation","weekend","work","workout"]
vector_result = pd.DataFrame(columns = cols_target, index = X_test.index)

for label in cols_target:
    print('... Processing {}'.format(label))
    train_y = y_train[label]
    logreg.fit(train_vector, train_y)
    pred_train = logreg.predict(train_vector)
    print('Training accuracy is {}'.format(accuracy_score(train_y, pred_train)))
    
    test_y_prob = logreg.predict_proba(test_vector)[:,1]
    test_y = y_test[label]
    pred_test = logreg.predict(test_vector)
    print('Testing accuracy is {}'.format(accuracy_score(test_y, pred_test)))
    vector_result[label] = test_y_prob

... Processing cold weather
Training accuracy is 0.9785657581370732
Testing accuracy is 0.9679012345679012
... Processing day to night
Training accuracy is 0.8936226514951046
Testing accuracy is 0.8419753086419753
... Processing night out
Training accuracy is 0.9036782217517861
Testing accuracy is 0.8450617283950618
... Processing vacation


/Users/bingruxue/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/bingruxue/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/module

Training accuracy is 0.9367557554908706
Testing accuracy is 0.8783950617283951
... Processing weekend
Training accuracy is 0.9050013231013495
Testing accuracy is 0.8265432098765432
... Processing work
Training accuracy is 0.9073829055305637
Testing accuracy is 0.85
... Processing workout
Training accuracy is 0.992590632442445
Testing accuracy is 0.9876543209876543


/Users/bingruxue/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [203]:
vector_result

,cold weather,day to night,night out,vacation,weekend,work,workout
id,,,,,,,
01e4ed7cp5h8mjkn93n1n30w8d01e4ed7cpdfeppkcq7dtkeet08,0.074446,0.069683,0.098187,0.755248,0.618033,0.573302,0.000787
01dtatf91fkbx2rw9k53jd1dq901dtatf91qzh744fekqw5z364e,0.000039,0.987312,0.334171,0.127344,0.508006,0.109986,0.000293
01e2kxsgamtvks4rrnxmymygzz01e2kxsznrcmyvgnjehk5sawf7,0.030449,0.108416,0.112304,0.845284,0.862245,0.132884,0.001707
01e4eaztq7jc3chps1bs500tdb01e4eaztqhg69v9344gagstty9,0.257677,0.902956,0.281858,0.119239,0.244321,0.788180,0.002000
01e2kzsh4mdqmrd79yx26vrz1f01e2kzsh4zmzcd1xhyxhtqt3z0,0.042159,0.991323,0.695139,0.022445,0.983454,0.130238,0.002539
...,...,...,...,...,...,...,...
01e2m40trb3jfmw8yqrm9kwqs901e2m42g19qp297dh2c50eat3f,0.006728,0.951990,0.024292,0.023188,0.055082,0.939382,0.000918
01e1jkxpbj02k4e8132x4x8w2c01e1jkxpbxkrqgb2mmbgh4wca8,0.001144,0.985155,0.160806,0.000419,0.727880,0.125507,0.000347
01e4rtwhb4zaagswpfd3awc4za01e4rtwhbe5mw2133e68x690gg,0.000326,0.645444,0.261826,0.104271,0.967929,0.285338,0.000671


_____

## 5. Combine embedding model and vector model together

In [282]:
# Get weighted-avg predicted probability from 2 models
combined_result = pd.DataFrame(index=vector_result.index,
                              columns=cols_target)
for label in cols_target:
    combined_result[label] = 0.2*embedding_result[label]+0.8*vector_result[label]

combined_result

,cold weather,day to night,night out,vacation,weekend,work,workout
id,,,,,,,
01e4ed7cp5h8mjkn93n1n30w8d01e4ed7cpdfeppkcq7dtkeet08,0.070545,0.194055,0.127166,0.640789,0.652261,0.525875,0.009142
01dtatf91fkbx2rw9k53jd1dq901dtatf91qzh744fekqw5z364e,0.011019,0.928158,0.315953,0.138466,0.564239,0.155223,0.008747
01e2kxsgamtvks4rrnxmymygzz01e2kxsznrcmyvgnjehk5sawf7,0.035348,0.225041,0.138460,0.712818,0.847630,0.173541,0.009878
01e4eaztq7jc3chps1bs500tdb01e4eaztqhg69v9344gagstty9,0.217130,0.860673,0.274103,0.131982,0.353291,0.697778,0.010113
01e2kzsh4mdqmrd79yx26vrz1f01e2kzsh4zmzcd1xhyxhtqt3z0,0.044716,0.931367,0.604727,0.054546,0.944597,0.171424,0.010543
...,...,...,...,...,...,...,...
01e2m40trb3jfmw8yqrm9kwqs901e2m42g19qp297dh2c50eat3f,0.016371,0.899900,0.068050,0.055141,0.201900,0.818740,0.009246
01e1jkxpbj02k4e8132x4x8w2c01e1jkxpbxkrqgb2mmbgh4wca8,0.011903,0.926432,0.177261,0.036926,0.740139,0.167639,0.008790
01e4rtwhb4zaagswpfd3awc4za01e4rtwhbe5mw2133e68x690gg,0.011249,0.654664,0.258077,0.120008,0.932178,0.295504,0.009049


In [283]:
# Final Decision Threshold:
## If a product's probability of belonging to an occasion is >0.5, assign occasion value
## If none of occasion probability is >0.5 for a product, assign with occasion with highest probability

def decision(probs):
    if sum(probs>0.5)>0:
        probs[probs > 0.5] = 1
        probs[probs <= 0.5] = 0
    else:
        probs[probs == np.max(probs)] = 1
        probs[probs != np.max(probs)] = 0
    return probs

In [284]:
combined_result.apply(decision, axis=1)

,cold weather,day to night,night out,vacation,weekend,work,workout
id,,,,,,,
01e4ed7cp5h8mjkn93n1n30w8d01e4ed7cpdfeppkcq7dtkeet08,0.0,0.0,0.0,1.0,1.0,1.0,0.0
01dtatf91fkbx2rw9k53jd1dq901dtatf91qzh744fekqw5z364e,0.0,1.0,0.0,0.0,1.0,0.0,0.0
01e2kxsgamtvks4rrnxmymygzz01e2kxsznrcmyvgnjehk5sawf7,0.0,0.0,0.0,1.0,1.0,0.0,0.0
01e4eaztq7jc3chps1bs500tdb01e4eaztqhg69v9344gagstty9,0.0,1.0,0.0,0.0,0.0,1.0,0.0
01e2kzsh4mdqmrd79yx26vrz1f01e2kzsh4zmzcd1xhyxhtqt3z0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
01e2m40trb3jfmw8yqrm9kwqs901e2m42g19qp297dh2c50eat3f,0.0,1.0,0.0,0.0,0.0,1.0,0.0
01e1jkxpbj02k4e8132x4x8w2c01e1jkxpbxkrqgb2mmbgh4wca8,0.0,1.0,0.0,0.0,1.0,0.0,0.0
01e4rtwhb4zaagswpfd3awc4za01e4rtwhbe5mw2133e68x690gg,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [285]:
combined_result.columns = [str(col) + '_pred' for col in combined_result.columns]
combined_result = combined_result.astype(int)
combined_result

,cold weather_pred,day to night_pred,night out_pred,vacation_pred,weekend_pred,work_pred,workout_pred
id,,,,,,,
01e4ed7cp5h8mjkn93n1n30w8d01e4ed7cpdfeppkcq7dtkeet08,0,0,0,1,1,1,0
01dtatf91fkbx2rw9k53jd1dq901dtatf91qzh744fekqw5z364e,0,1,0,0,1,0,0
01e2kxsgamtvks4rrnxmymygzz01e2kxsznrcmyvgnjehk5sawf7,0,0,0,1,1,0,0
01e4eaztq7jc3chps1bs500tdb01e4eaztqhg69v9344gagstty9,0,1,0,0,0,1,0
01e2kzsh4mdqmrd79yx26vrz1f01e2kzsh4zmzcd1xhyxhtqt3z0,0,1,1,0,1,0,0
...,...,...,...,...,...,...,...
01e2m40trb3jfmw8yqrm9kwqs901e2m42g19qp297dh2c50eat3f,0,1,0,0,0,1,0
01e1jkxpbj02k4e8132x4x8w2c01e1jkxpbxkrqgb2mmbgh4wca8,0,1,0,0,1,0,0
01e4rtwhb4zaagswpfd3awc4za01e4rtwhbe5mw2133e68x690gg,0,1,0,0,1,0,0


In [286]:
# ## Check final accuracy
# accuracy_score(y_test,combined_result)

In [287]:
y_test

,cold weather,day to night,night out,vacation,weekend,work,workout
id,,,,,,,
01e4ed7cp5h8mjkn93n1n30w8d01e4ed7cpdfeppkcq7dtkeet08,0,0,1,0,1,0,0
01dtatf91fkbx2rw9k53jd1dq901dtatf91qzh744fekqw5z364e,0,1,0,0,1,0,0
01e2kxsgamtvks4rrnxmymygzz01e2kxsznrcmyvgnjehk5sawf7,0,0,0,1,1,0,0
01e4eaztq7jc3chps1bs500tdb01e4eaztqhg69v9344gagstty9,1,1,0,0,0,1,0
01e2kzsh4mdqmrd79yx26vrz1f01e2kzsh4zmzcd1xhyxhtqt3z0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...
01e2m40trb3jfmw8yqrm9kwqs901e2m42g19qp297dh2c50eat3f,0,1,0,0,0,1,0
01e1jkxpbj02k4e8132x4x8w2c01e1jkxpbxkrqgb2mmbgh4wca8,0,1,0,0,1,0,0
01e4rtwhb4zaagswpfd3awc4za01e4rtwhbe5mw2133e68x690gg,0,0,1,0,0,0,0


In [288]:
compare = pd.concat([combined_result, y_test], axis=1, sort=False)
compare

,cold weather_pred,day to night_pred,night out_pred,vacation_pred,weekend_pred,work_pred,workout_pred,cold weather,day to night,night out,vacation,weekend,work,workout
id,,,,,,,,,,,,,,
01e4ed7cp5h8mjkn93n1n30w8d01e4ed7cpdfeppkcq7dtkeet08,0,0,0,1,1,1,0,0,0,1,0,1,0,0
01dtatf91fkbx2rw9k53jd1dq901dtatf91qzh744fekqw5z364e,0,1,0,0,1,0,0,0,1,0,0,1,0,0
01e2kxsgamtvks4rrnxmymygzz01e2kxsznrcmyvgnjehk5sawf7,0,0,0,1,1,0,0,0,0,0,1,1,0,0
01e4eaztq7jc3chps1bs500tdb01e4eaztqhg69v9344gagstty9,0,1,0,0,0,1,0,1,1,0,0,0,1,0
01e2kzsh4mdqmrd79yx26vrz1f01e2kzsh4zmzcd1xhyxhtqt3z0,0,1,1,0,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
01e2m40trb3jfmw8yqrm9kwqs901e2m42g19qp297dh2c50eat3f,0,1,0,0,0,1,0,0,1,0,0,0,1,0
01e1jkxpbj02k4e8132x4x8w2c01e1jkxpbxkrqgb2mmbgh4wca8,0,1,0,0,1,0,0,0,1,0,0,1,0,0
01e4rtwhb4zaagswpfd3awc4za01e4rtwhbe5mw2133e68x690gg,0,1,0,0,1,0,0,0,0,1,0,0,0,0


In [293]:
def accuracy(table):
    total_actual = len(compare)*7
    acc=0
    for index, row in table.iterrows():
        if row['cold weather_pred'] == row['cold weather']:
            acc+=1
        if row['day to night_pred'] == row['day to night']:
            acc+=1
        if row['night out_pred'] == row['night out']:
            acc+=1
        if row['vacation_pred'] == row['vacation']:
            acc+=1
        if row['weekend_pred'] == row['weekend']:
            acc+=1
        if row['work_pred'] == row['work']:
            acc+=1
        if row['workout_pred'] == row['workout']:
            acc+=1
    acc_rate = acc/total_actual
    return acc,acc_rate

In [294]:
accuracy(compare)

(10029, 0.8843915343915344)